In [184]:
import pandas as pd
import numpy as np

In [249]:
prolong_df = pd.read_csv('prolongations.csv')
financial_df = pd.read_csv('financial_data.csv')

In [250]:
# Все колонки, которые соответствуют месяцам, без служебных полей
shipment_cols = [c for c in financial_df.columns if c not in ['id', 'Причина дубля', 'Account']]

In [251]:
prolong_df['month'] = prolong_df['month'].str.capitalize()

In [252]:
# Все id проектов, у которых присутствует "стоп" или "end" в последний месяц или ранее
bad_id_mask = []
for idx, row in financial_df.merge(prolong_df[['id','month']], on='id', how='right').iterrows():
    cutoff = row['month']
    check_months = shipment_cols[:shipment_cols.index(cutoff) + 1]
    if any(str(row[c]).strip().lower() in ('стоп','end') for c in check_months):
        bad_id_mask.append(row['id'])
bad_ids = set(bad_id_mask)

In [253]:
financial_df = financial_df[~financial_df['id'].isin(bad_ids)]
prolong_df = prolong_df[~prolong_df['id'].isin(bad_ids)]

In [ ]:
# Помечаем отгрузки, где есть "в ноль"
zero_flags = (financial_df[shipment_cols]
              .map(lambda x: str(x).strip().lower() == 'в ноль')
              .groupby(financial_df['id'])
              .any()) 

In [ ]:
# Функция, преобразовывающая строки в числовой формат
def str_to_float(x):
    s = str(x).strip().lower()
    if s == 'в ноль':
        return np.nan
    s = s.replace('\u00A0','').replace('"', '').replace(' ','').replace(',','.')
    try:
        return float(s)
    except:
        return np.nan

In [ ]:
# Преобразованная в числа информация
parsed_financial_df = (financial_df[shipment_cols]
                       .map(str_to_float)
                       .groupby(financial_df['id'])
                       .sum()) 

In [ ]:
# Сбрасываем значения в nan там, где нет других оплат и стоит "в ноль"
masked = parsed_financial_df.mask((parsed_financial_df == 0) & zero_flags)

In [ ]:
# Заполняем сброшенные в nan данные данными за прошлый месяц (в несброшенных данных уже стоит 0 или другая сумма)
financial_df_filled = masked.ffill(axis=1).fillna(0.0)

In [259]:
# Объединяем информацию о пролонгациях с финансовыми данными
merged_info_df = pd.merge(
    prolong_df.rename(columns={'AM': 'Аккаунт-менеджер'}),
    financial_df_filled.reset_index(),
    on='id',
    how='left'
).fillna(0.0)

In [260]:
# Карта номера месяца по его названию
month_map = {
    'Январь': 1,
    'Февраль': 2,
    'Март': 3,
    'Апрель': 4,
    'Май': 5,
    'Июнь': 6,
    'Июль': 7,
    'Август': 8,
    'Сентябрь': 9,
    'Октябрь': 10,
    'Ноябрь': 11,
    'Декабрь': 12
}

# Обратная карта для формирования имени месяца по его номеру
reverse_month_map = {v: k for k, v in month_map.items()}

In [261]:
# Функция, которая для строки вида "Май 2023" возвращает предыдущий месяц, например "Апрель 2023"
def prev_month(month_year_str: str) -> str:    
    month, year = month_year_str.split()
    month_index = month_map[month]
    if month_index == 1:
        return f"{reverse_month_map[12]} {int(year) - 1}"
    return f"{reverse_month_map[month_index - 1]} {year}"

In [262]:
# Функция, которая для строки вида "Май 2023" возвращает месяц за два шага назад, например "Март 2023"
def prev_two_months(month_year_str: str) -> str:    
    month, year = month_year_str.split()
    month_index = month_map[month]
    match month_index:
        case 1:
            return f"{reverse_month_map[11]} {int(year) - 1}"
        case 2:
            return f"{reverse_month_map[12]} {int(year) - 1}"
        case _:
            return f"{reverse_month_map[month_index - 2]} {int(year)}"    

In [263]:
managers = sorted(merged_info_df['Аккаунт-менеджер'].unique().tolist()) + ['Весь отдел']

In [264]:
months_2023 = [f"{reverse_month_map[i]} 2023" for i in range(1, 13)]

In [265]:
# Формируем аналитику по месячным коэффициентам пролонгации
results = []

for mgr in managers:
    sub_df = merged_info_df if mgr == 'Весь отдел' else merged_info_df[merged_info_df['Аккаунт-менеджер'] == mgr]
    for m in months_2023:
        previous_month  = prev_month(m) # предыдущий месяц
        month_before_previous = prev_two_months(m) # месяц за два шага назад

        # Первомесячная пролонгация
        denom1 = sub_df.loc[sub_df['month'] == previous_month, previous_month].sum()
        num1 = sub_df.loc[(sub_df['month'] == previous_month) & (sub_df[m] > 0), m].sum()
        rate1 = num1 / denom1 if denom1 > 0 else float('nan')

        # Второмесячная пролонгация только для тех проектов, которые не пролонгировались в первый месяц
        denom2 = sub_df.loc[(sub_df['month'] == month_before_previous) & (sub_df[previous_month] == 0), month_before_previous].sum()
        num2 = sub_df.loc[(sub_df['month'] == month_before_previous) & (sub_df[previous_month] == 0) & (sub_df[m] > 0), m].sum()
        rate2 = num2 / denom2 if denom2 > 0 else float('nan')

        results.append({
            'Аккаунт-менеджер': mgr,
            'Месяц': m,
            'Коэффициент первомесячной пролонгации': rate1,
            'Коэффициент второмесячной пролонгации': rate2
        })

In [266]:
monthly_rates_df = pd.DataFrame(results)

In [ ]:
# Формируем аналитику по годовым коэффициентам пролонгации
annual_results = []

for mgr in managers:
    sub_df = merged_info_df if mgr == 'Весь отдел' else merged_info_df[merged_info_df['Аккаунт-менеджер'] == mgr]

    # Накопительные суммы по первому и второму месяцам
    total_denom1 = 0.0
    total_num1 = 0.0
    total_denom2 = 0.0
    total_num2 = 0.0

    for m in months_2023:
        previous_month  = prev_month(m)
        month_before_previous = prev_two_months(m)

        # Знаменатель и числитель для первомесячной пролонгации
        denom1 = sub_df.loc[sub_df['month'] == previous_month, previous_month].sum()
        num1 = sub_df.loc[(sub_df['month'] == previous_month) & (sub_df[m] > 0), m].sum()

        # Для второмесячной
        denom2 = sub_df.loc[(sub_df['month'] == month_before_previous) & (sub_df[previous_month] == 0), month_before_previous].sum()
        num2 = sub_df.loc[(sub_df['month'] == month_before_previous) & (sub_df[previous_month] == 0) & (sub_df[m] > 0), m].sum()

        total_denom1 += denom1
        total_num1 += num1
        total_denom2 += denom2
        total_num2 += num2

    annual_first_rate = total_num1 / total_denom1 if total_denom1 > 0 else float('nan')
    annual_second_rate = total_num2 / total_denom2  if total_denom2 > 0 else float('nan')

    annual_results.append({
        'Аккаунт-менеджер': mgr,
        'Годовой коэффициент первомесячной пролонгации': annual_first_rate,
        'Годовой коэффициент второмесячной пролонгации': annual_second_rate
    })

annual_rates_df = pd.DataFrame(annual_results)

In [268]:
monthly_rates_df.to_csv("monthly_rates.csv")
annual_rates_df.to_csv("annual_rates.csv")